# Análisis de Red de Tiendas RetailNow

## Descripción del Proyecto
Este notebook contiene el análisis detallado del rendimiento de las diferentes sucursales de RetailNow en varias ciudades. Utilizaremos **Pandas** para la manipulación de datos y **Numpy** para cálculos estadísticos y simulaciones.

### Objetivos:
- Analizar datos de ventas, inventarios y satisfacción del cliente
- Identificar tiendas con rendimiento crítico
- Realizar proyecciones de ventas futuras
- Proporcionar recomendaciones estratégicas

## 1. Importación de Librerías

In [ ]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
import warnings
from datetime import datetime

# Configuración para mejorar la visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
warnings.filterwarnings('ignore')

print("✅ Librerías importadas correctamente")
print(f"📊 Pandas versión: {pd.__version__}")
print(f"🔢 Numpy versión: {np.__version__}")

## 2. Carga y Procesamiento de Datos

In [ ]:
# Definir rutas de los archivos CSV
# Nota: Adaptando las rutas al entorno local
ruta_ventas = r'/workspace/ventas.csv'
ruta_inventarios = r'/workspace/inventarios.csv'
ruta_satisfaccion = r'/workspace/satisfaccion.csv'

try:
    # Cargar los datos en DataFrames
    df_ventas = pd.read_csv(ruta_ventas)
    df_inventarios = pd.read_csv(ruta_inventarios)
    df_satisfaccion = pd.read_csv(ruta_satisfaccion)
    
    print("✅ Datos cargados exitosamente")
    print(f"📈 Registros de ventas: {len(df_ventas)}")
    print(f"📦 Registros de inventarios: {len(df_inventarios)}")
    print(f"😊 Registros de satisfacción: {len(df_satisfaccion)}")
    
except FileNotFoundError as e:
    print(f"❌ Error: No se pudo encontrar el archivo {e}")
except Exception as e:
    print(f"❌ Error inesperado: {e}")

### 2.1 Exploración Inicial de los Datos

In [ ]:
# Explorar la estructura de los DataFrames
print("=" * 60)
print("📊 ESTRUCTURA DEL DATASET DE VENTAS")
print("=" * 60)
print(f"Forma del DataFrame: {df_ventas.shape}")
print(f"Columnas: {list(df_ventas.columns)}")
print("\nPrimeras 5 filas:")
display(df_ventas.head())

print("\n" + "=" * 60)
print("📦 ESTRUCTURA DEL DATASET DE INVENTARIOS")
print("=" * 60)
print(f"Forma del DataFrame: {df_inventarios.shape}")
print(f"Columnas: {list(df_inventarios.columns)}")
print("\nPrimeras 5 filas:")
display(df_inventarios.head())

print("\n" + "=" * 60)
print("😊 ESTRUCTURA DEL DATASET DE SATISFACCIÓN")
print("=" * 60)
print(f"Forma del DataFrame: {df_satisfaccion.shape}")
print(f"Columnas: {list(df_satisfaccion.columns)}")
print("\nPrimeras 5 filas:")
display(df_satisfaccion.head())

### 2.2 Limpieza de Datos

In [ ]:
# Verificar valores nulos antes de la limpieza
print("🔍 VALORES NULOS ANTES DE LA LIMPIEZA")
print("=" * 50)
print("Ventas:")
print(df_ventas.isnull().sum())
print("\nInventarios:")
print(df_inventarios.isnull().sum())
print("\nSatisfacción:")
print(df_satisfaccion.isnull().sum())

# Limpiar datos eliminando filas con valores nulos
df_ventas_clean = df_ventas.dropna()
df_inventarios_clean = df_inventarios.dropna()
df_satisfaccion_clean = df_satisfaccion.dropna()

print("\n✅ DATOS DESPUÉS DE LA LIMPIEZA")
print("=" * 50)
print(f"Ventas: {len(df_ventas)} → {len(df_ventas_clean)} registros")
print(f"Inventarios: {len(df_inventarios)} → {len(df_inventarios_clean)} registros")
print(f"Satisfacción: {len(df_satisfaccion)} → {len(df_satisfaccion_clean)} registros")

# Verificar tipos de datos
print("\n📋 TIPOS DE DATOS")
print("=" * 30)
print("Ventas:")
print(df_ventas_clean.dtypes)
print("\nInventarios:")
print(df_inventarios_clean.dtypes)
print("\nSatisfacción:")
print(df_satisfaccion_clean.dtypes)

## 3. Exploración y Análisis de Datos con Pandas

### 3.1 Análisis de Ventas

In [ ]:
# Calcular ingresos totales por venta (cantidad × precio)
df_ventas_clean['ingresos_venta'] = df_ventas_clean['cantidad_vendida'] * df_ventas_clean['precio_unitario']

# Calcular ventas totales por tienda
ventas_por_tienda = df_ventas_clean.groupby('tienda_id').agg({
    'cantidad_vendida': 'sum',
    'ingresos_venta': 'sum',
    'producto_id': 'count'  # Número de transacciones
}).rename(columns={'producto_id': 'num_transacciones'})

ventas_por_tienda['ingreso_promedio_transaccion'] = ventas_por_tienda['ingresos_venta'] / ventas_por_tienda['num_transacciones']

print("💰 VENTAS TOTALES POR TIENDA")
print("=" * 50)
display(ventas_por_tienda.round(2))

# Calcular ventas por categoría
ventas_por_categoria = df_ventas_clean.groupby('categoria').agg({
    'cantidad_vendida': 'sum',
    'ingresos_venta': 'sum'
})

print("\n📊 VENTAS POR CATEGORÍA")
print("=" * 40)
display(ventas_por_categoria.round(2))

# Resumen estadístico de ventas
print("\n📈 RESUMEN ESTADÍSTICO DE INGRESOS")
print("=" * 45)
display(df_ventas_clean['ingresos_venta'].describe())

### 3.2 Análisis de Inventarios y Rotación

In [ ]:
# Calcular rotación de inventarios
# Primero, obtener ventas totales por tienda y producto
ventas_por_producto = df_ventas_clean.groupby(['tienda_id', 'producto_id'])['cantidad_vendida'].sum().reset_index()

# Unir con datos de inventario
inventario_con_ventas = df_inventarios_clean.merge(
    ventas_por_producto, 
    on=['tienda_id', 'producto_id'], 
    how='left'
)

# Llenar valores nulos con 0 (productos sin ventas)
inventario_con_ventas['cantidad_vendida'] = inventario_con_ventas['cantidad_vendida'].fillna(0)

# Calcular rotación de inventario (ventas / stock disponible)
inventario_con_ventas['rotacion_inventario'] = (
    inventario_con_ventas['cantidad_vendida'] / inventario_con_ventas['stock_disponible']
) * 100

# Calcular rotación promedio por tienda
rotacion_por_tienda = inventario_con_ventas.groupby('tienda_id').agg({
    'rotacion_inventario': 'mean',
    'stock_disponible': 'sum',
    'cantidad_vendida': 'sum'
})

rotacion_por_tienda['rotacion_total'] = (
    rotacion_por_tienda['cantidad_vendida'] / rotacion_por_tienda['stock_disponible']
) * 100

print("🔄 ROTACIÓN DE INVENTARIOS POR TIENDA")
print("=" * 50)
display(rotacion_por_tienda.round(2))

# Identificar tiendas con inventarios críticos (< 10% de rotación)
tiendas_criticas = rotacion_por_tienda[rotacion_por_tienda['rotacion_total'] < 10]

print("\n⚠️  TIENDAS CON INVENTARIOS CRÍTICOS (< 10% rotación)")
print("=" * 60)
if len(tiendas_criticas) > 0:
    display(tiendas_criticas.round(2))
else:
    print("✅ No hay tiendas con inventarios críticos")

# Productos con menor rotación
productos_baja_rotacion = inventario_con_ventas[inventario_con_ventas['rotacion_inventario'] < 5]
print(f"\n📉 PRODUCTOS CON BAJA ROTACIÓN (< 5%): {len(productos_baja_rotacion)} productos")
if len(productos_baja_rotacion) > 0:
    display(productos_baja_rotacion[['tienda_id', 'producto', 'categoria', 'rotacion_inventario']].head(10))

### 3.3 Análisis de Satisfacción del Cliente

In [ ]:
# Calcular satisfacción promedio por tienda
satisfaccion_por_tienda = df_satisfaccion_clean.groupby('tienda_id').agg({
    'puntuacion_satisfaccion': ['mean', 'std', 'count', 'min', 'max']
}).round(2)

# Aplanar nombres de columnas
satisfaccion_por_tienda.columns = ['satisfaccion_promedio', 'desviacion_std', 'num_encuestas', 'min_puntuacion', 'max_puntuacion']

print("😊 SATISFACCIÓN DEL CLIENTE POR TIENDA")
print("=" * 50)
display(satisfaccion_por_tienda)

# Identificar tiendas con baja satisfacción (< 60%)
tiendas_baja_satisfaccion = satisfaccion_por_tienda[satisfaccion_por_tienda['satisfaccion_promedio'] < 60]

print("\n⚠️  TIENDAS CON BAJA SATISFACCIÓN (< 60%)")
print("=" * 50)
if len(tiendas_baja_satisfaccion) > 0:
    display(tiendas_baja_satisfaccion)
    
    print("\n📋 RECOMENDACIONES PARA TIENDAS CON BAJA SATISFACCIÓN:")
    print("=" * 60)
    for tienda in tiendas_baja_satisfaccion.index:
        puntuacion = tiendas_baja_satisfaccion.loc[tienda, 'satisfaccion_promedio']
        print(f"🏪 {tienda}: Puntuación {puntuacion:.1f}%")
        print("   • Mejorar capacitación del personal")
        print("   • Revisar procesos de atención al cliente")
        print("   • Implementar programa de mejora de calidad")
        print("   • Monitorear inventarios para evitar faltantes")
        print()
else:
    print("✅ Todas las tiendas tienen satisfacción aceptable (≥ 60%)")

# Distribución de satisfacción
print("\n📊 DISTRIBUCIÓN DE SATISFACCIÓN")
print("=" * 40)
rangos_satisfaccion = pd.cut(df_satisfaccion_clean['puntuacion_satisfaccion'], 
                           bins=[0, 40, 60, 80, 100], 
                           labels=['Muy Baja (0-40)', 'Baja (41-60)', 'Buena (61-80)', 'Excelente (81-100)'])
distribucion = rangos_satisfaccion.value_counts()
display(distribucion)

## 4. Cálculos Estadísticos con Numpy

### 4.1 Estadísticas de Ventas con Numpy

In [ ]:
# Convertir datos de ventas a arrays de Numpy
ingresos_array = df_ventas_clean['ingresos_venta'].to_numpy()
cantidades_array = df_ventas_clean['cantidad_vendida'].to_numpy()

# Calcular estadísticas con Numpy
mediana_ingresos = np.median(ingresos_array)
desviacion_std_ingresos = np.std(ingresos_array)
media_ingresos = np.mean(ingresos_array)
percentil_25 = np.percentile(ingresos_array, 25)
percentil_75 = np.percentile(ingresos_array, 75)

print("🔢 ESTADÍSTICAS DE INGRESOS CON NUMPY")
print("=" * 45)
print(f"💰 Media de ingresos: ${media_ingresos:.2f}")
print(f"📊 Mediana de ingresos: ${mediana_ingresos:.2f}")
print(f"📈 Desviación estándar: ${desviacion_std_ingresos:.2f}")
print(f"📉 Percentil 25: ${percentil_25:.2f}")
print(f"📈 Percentil 75: ${percentil_75:.2f}")
print(f"📊 Rango intercuartílico: ${percentil_75 - percentil_25:.2f}")

# Estadísticas por tienda usando Numpy
print("\n🏪 ESTADÍSTICAS POR TIENDA (NUMPY)")
print("=" * 40)
for tienda in df_ventas_clean['tienda_id'].unique():
    ingresos_tienda = df_ventas_clean[df_ventas_clean['tienda_id'] == tienda]['ingresos_venta'].to_numpy()
    
    mediana_tienda = np.median(ingresos_tienda)
    std_tienda = np.std(ingresos_tienda)
    total_tienda = np.sum(ingresos_tienda)
    
    print(f"{tienda}: Total=${total_tienda:.2f}, Mediana=${mediana_tienda:.2f}, Std=${std_tienda:.2f}")

### 4.2 Simulación de Proyecciones Futuras con Numpy

In [ ]:
# Establecer semilla para reproducibilidad
np.random.seed(42)

# Parámetros para la simulación
num_simulaciones = 1000
num_meses = 12

# Calcular estadísticas base de ventas actuales
ingresos_totales_actuales = ventas_por_tienda['ingresos_venta'].to_numpy()
media_ingresos_tienda = np.mean(ingresos_totales_actuales)
std_ingresos_tienda = np.std(ingresos_totales_actuales)

print("🔮 SIMULACIÓN DE PROYECCIONES DE VENTAS FUTURAS")
print("=" * 55)
print(f"📊 Base de cálculo:")
print(f"   Media actual: ${media_ingresos_tienda:.2f}")
print(f"   Desviación estándar: ${std_ingresos_tienda:.2f}")
print(f"   Número de simulaciones: {num_simulaciones:,}")
print(f"   Período de proyección: {num_meses} meses")

# Simular ventas futuras con diferentes escenarios
# Escenario 1: Crecimiento conservador (2% mensual)
crecimiento_conservador = 1.02
proyecciones_conservadoras = []

for mes in range(num_meses):
    # Aplicar crecimiento y variabilidad
    factor_crecimiento = crecimiento_conservador ** mes
    media_mes = media_ingresos_tienda * factor_crecimiento
    
    # Generar simulaciones con distribución normal
    simulaciones_mes = np.random.normal(media_mes, std_ingresos_tienda, num_simulaciones)
    # Asegurar que no hay valores negativos
    simulaciones_mes = np.maximum(simulaciones_mes, 0)
    
    proyecciones_conservadoras.append(simulaciones_mes)

proyecciones_conservadoras = np.array(proyecciones_conservadoras)

# Escenario 2: Crecimiento optimista (5% mensual)
crecimiento_optimista = 1.05
proyecciones_optimistas = []

for mes in range(num_meses):
    factor_crecimiento = crecimiento_optimista ** mes
    media_mes = media_ingresos_tienda * factor_crecimiento
    
    simulaciones_mes = np.random.normal(media_mes, std_ingresos_tienda * 1.2, num_simulaciones)
    simulaciones_mes = np.maximum(simulaciones_mes, 0)
    
    proyecciones_optimistas.append(simulaciones_mes)

proyecciones_optimistas = np.array(proyecciones_optimistas)

# Escenario 3: Escenario pesimista (-1% mensual)
crecimiento_pesimista = 0.99
proyecciones_pesimistas = []

for mes in range(num_meses):
    factor_crecimiento = crecimiento_pesimista ** mes
    media_mes = media_ingresos_tienda * factor_crecimiento
    
    simulaciones_mes = np.random.normal(media_mes, std_ingresos_tienda * 0.8, num_simulaciones)
    simulaciones_mes = np.maximum(simulaciones_mes, 0)
    
    proyecciones_pesimistas.append(simulaciones_mes)

proyecciones_pesimistas = np.array(proyecciones_pesimistas)

print("\n📈 RESULTADOS DE LAS SIMULACIONES")
print("=" * 40)

# Analizar resultados por escenario
escenarios = {
    'Conservador (+2% mensual)': proyecciones_conservadoras,
    'Optimista (+5% mensual)': proyecciones_optimistas,
    'Pesimista (-1% mensual)': proyecciones_pesimistas
}

for nombre_escenario, proyecciones in escenarios.items():
    print(f"\n🎯 ESCENARIO {nombre_escenario.upper()}")
    print("-" * 50)
    
    # Estadísticas del último mes (mes 12)
    ultimo_mes = proyecciones[-1]
    
    media_final = np.mean(ultimo_mes)
    mediana_final = np.median(ultimo_mes)
    std_final = np.std(ultimo_mes)
    percentil_5 = np.percentile(ultimo_mes, 5)
    percentil_95 = np.percentile(ultimo_mes, 95)
    
    print(f"💰 Ingreso promedio proyectado (mes 12): ${media_final:.2f}")
    print(f"📊 Mediana proyectada: ${mediana_final:.2f}")
    print(f"📈 Desviación estándar: ${std_final:.2f}")
    print(f"📉 Rango de confianza 90%: ${percentil_5:.2f} - ${percentil_95:.2f}")
    
    # Calcular crecimiento total
    crecimiento_total = ((media_final - media_ingresos_tienda) / media_ingresos_tienda) * 100
    print(f"📊 Crecimiento total proyectado: {crecimiento_total:.1f}%")

# Resumen comparativo
print("\n📋 RESUMEN COMPARATIVO DE ESCENARIOS (MES 12)")
print("=" * 60)
resumen_data = []
for nombre, proyecciones in escenarios.items():
    ultimo_mes = proyecciones[-1]
    resumen_data.append({
        'Escenario': nombre,
        'Media': f"${np.mean(ultimo_mes):.2f}",
        'Mediana': f"${np.median(ultimo_mes):.2f}",
        'Desv_Std': f"${np.std(ultimo_mes):.2f}",
        'Min': f"${np.min(ultimo_mes):.2f}",
        'Max': f"${np.max(ultimo_mes):.2f}"
    })

df_resumen = pd.DataFrame(resumen_data)
display(df_resumen)

## 5. Análisis Integrado y Recomendaciones

In [ ]:
# Crear un análisis integrado combinando todos los datos
print("🔍 ANÁLISIS INTEGRADO DE RENDIMIENTO POR TIENDA")
print("=" * 60)

# Combinar todos los análisis en un DataFrame integrado
analisis_integrado = ventas_por_tienda.copy()
analisis_integrado = analisis_integrado.merge(
    rotacion_por_tienda[['rotacion_total']], 
    left_index=True, 
    right_index=True
)
analisis_integrado = analisis_integrado.merge(
    satisfaccion_por_tienda[['satisfaccion_promedio']], 
    left_index=True, 
    right_index=True
)

# Crear categorías de rendimiento
def categorizar_rendimiento(row):
    score = 0
    
    # Puntuación por ingresos (25% del total)
    if row['ingresos_venta'] > analisis_integrado['ingresos_venta'].quantile(0.75):
        score += 25
    elif row['ingresos_venta'] > analisis_integrado['ingresos_venta'].quantile(0.5):
        score += 15
    elif row['ingresos_venta'] > analisis_integrado['ingresos_venta'].quantile(0.25):
        score += 10
    
    # Puntuación por rotación de inventario (25% del total)
    if row['rotacion_total'] > 15:
        score += 25
    elif row['rotacion_total'] > 10:
        score += 15
    elif row['rotacion_total'] > 5:
        score += 10
    
    # Puntuación por satisfacción (50% del total)
    if row['satisfaccion_promedio'] >= 80:
        score += 50
    elif row['satisfaccion_promedio'] >= 70:
        score += 35
    elif row['satisfaccion_promedio'] >= 60:
        score += 20
    elif row['satisfaccion_promedio'] >= 50:
        score += 10
    
    if score >= 80:
        return 'Excelente'
    elif score >= 60:
        return 'Bueno'
    elif score >= 40:
        return 'Regular'
    else:
        return 'Crítico'

analisis_integrado['categoria_rendimiento'] = analisis_integrado.apply(categorizar_rendimiento, axis=1)

# Mostrar análisis integrado
display(analisis_integrado.round(2))

# Resumen por categoría de rendimiento
print("\n📊 DISTRIBUCIÓN DE TIENDAS POR RENDIMIENTO")
print("=" * 50)
distribucion_rendimiento = analisis_integrado['categoria_rendimiento'].value_counts()
for categoria, cantidad in distribucion_rendimiento.items():
    porcentaje = (cantidad / len(analisis_integrado)) * 100
    print(f"{categoria}: {cantidad} tiendas ({porcentaje:.1f}%)")

print("\n🎯 RECOMENDACIONES ESTRATÉGICAS POR TIENDA")
print("=" * 55)

for tienda in analisis_integrado.index:
    datos = analisis_integrado.loc[tienda]
    categoria = datos['categoria_rendimiento']
    
    print(f"\n🏪 {tienda} - Rendimiento: {categoria}")
    print("-" * 40)
    
    if categoria == 'Excelente':
        print("✅ Mantener las buenas prácticas actuales")
        print("✅ Considerar como modelo para otras tiendas")
        print("✅ Explorar oportunidades de expansión")
    
    elif categoria == 'Bueno':
        print("📈 Identificar áreas específicas de mejora")
        print("📈 Implementar mejores prácticas de tiendas excelentes")
        
    elif categoria == 'Regular':
        print("⚠️  Requiere atención inmediata")
        if datos['satisfaccion_promedio'] < 70:
            print("⚠️  Prioridad: Mejorar satisfacción del cliente")
        if datos['rotacion_total'] < 10:
            print("⚠️  Prioridad: Optimizar gestión de inventarios")
        if datos['ingresos_venta'] < analisis_integrado['ingresos_venta'].median():
            print("⚠️  Prioridad: Estrategias para aumentar ventas")
    
    else:  # Crítico
        print("🚨 REQUIERE INTERVENCIÓN URGENTE")
        print("🚨 Plan de mejora integral necesario")
        print("🚨 Considerar reestructuración o cierre temporal")

print("\n" + "=" * 70)
print("📋 CONCLUSIONES GENERALES DEL ANÁLISIS")
print("=" * 70)

# Estadísticas generales
ingresos_totales = analisis_integrado['ingresos_venta'].sum()
satisfaccion_promedio_general = analisis_integrado['satisfaccion_promedio'].mean()
rotacion_promedio_general = analisis_integrado['rotacion_total'].mean()

print(f"💰 Ingresos totales de la red: ${ingresos_totales:,.2f}")
print(f"😊 Satisfacción promedio de la red: {satisfaccion_promedio_general:.1f}%")
print(f"🔄 Rotación promedio de inventarios: {rotacion_promedio_general:.1f}%")
print(f"🏪 Total de tiendas analizadas: {len(analisis_integrado)}")

# Identificar mejores y peores performers
mejor_tienda = analisis_integrado.loc[analisis_integrado['ingresos_venta'].idxmax()]
peor_tienda = analisis_integrado.loc[analisis_integrado['ingresos_venta'].idxmin()]

print(f"\n🏆 Mejor tienda por ingresos: {mejor_tienda.name} (${mejor_tienda['ingresos_venta']:,.2f})")
print(f"📉 Tienda con menores ingresos: {peor_tienda.name} (${peor_tienda['ingresos_venta']:,.2f})")

print("\n🎯 PRÓXIMOS PASOS RECOMENDADOS:")
print("1. Implementar plan de mejora para tiendas críticas")
print("2. Replicar mejores prácticas de tiendas excelentes")
print("3. Monitorear KPIs mensualmente")
print("4. Revisar estrategias de inventario y satisfacción")
print("5. Considerar las proyecciones de crecimiento para planificación")

## 6. Conclusiones y Próximos Pasos

### Resumen del Análisis
Este análisis ha proporcionado una visión integral del rendimiento de la red de tiendas RetailNow, utilizando:

- **Pandas** para manipulación y análisis de datos
- **Numpy** para cálculos estadísticos y simulaciones
- **Análisis multidimensional** combinando ventas, inventarios y satisfacción

### Hallazgos Clave
1. **Rendimiento Variable**: Las tiendas muestran diferentes niveles de rendimiento
2. **Correlación**: Existe relación entre satisfacción del cliente y ventas
3. **Oportunidades**: Identificadas áreas específicas de mejora por tienda
4. **Proyecciones**: Simulaciones futuras ayudan en la planificación estratégica

### Valor del Análisis
- **Decisiones basadas en datos**: Recomendaciones específicas por tienda
- **Identificación proactiva**: Detección temprana de problemas
- **Optimización de recursos**: Enfoque dirigido en áreas críticas
- **Planificación futura**: Proyecciones para diferentes escenarios

---
*Análisis realizado con Python 3.12, Pandas y Numpy*